Importamos librerias

In [3]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV
from matplotlib import pyplot

Función One Hot Enconding

In [1]:
# Archivo utilizado para crear la función de One Hot Encoding
############################################################# 

# Función auxiliar: Toma un df de variables categoricas y selección el top_x de categorias
def one_hot_top_x(cat_df, variable,top_x_labels):
    for label in top_x_labels:
        cat_df[str(variable)+'_'+str(label)] = np.where(cat_df[variable]==label,1,0)

# Función One Hot Encoder: Toma un DF y le hace una transformación de One Hot Encoder para con las categorias más repetidas 
def one_hot_encoder(df,top_x):
    # Separo variables categoricas de númericas
    num_cols = df.select_dtypes(include=['float64', 'int']).columns.to_list()
    cat_cols = df.select_dtypes(include=['object', 'category']).columns.to_list()
    
    # Armo un DF solo con variables categoricas
    cat_df = df.drop(num_cols,axis=1)
    # Armo un DF solo con variables categoricas
    num_df = df.drop(cat_cols,axis=1)
   
    # Itero entre las variables categoricas y me quedo con el top 10 variables
    for variable in cat_cols:
        top_10 = [x for x in cat_df[variable].value_counts().sort_values(ascending=False).head(top_x).index]
        one_hot_top_x(cat_df,variable,top_10)
    
    cat_df = cat_df.drop(cat_cols,axis=1)
    df = pd.concat([cat_df,num_df], axis =1)
    
    return df

Importamos datos

In [6]:

# Levantamos los datos procesados
datos_meli = pd.read_excel('data_ready_to_model.xlsx')
datos_meli['type'] = 'meli'

# Uno los DF
frames = [datos_meli, datos_privados]
datos = pd.concat(frames)

# Los paso por One Hot Encoder
datos = one_hot_encoder(datos,100)

# Separo los datos de Meli
datos_meli = datos[(datos['type_meli'] == 1)] 
datos_meli = datos_meli.drop('type_meli',axis=1)

C:\Users\LUCIA~1.LOP\AppData\Local\Temp/ipykernel_23324/187505526.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  cat_df[str(variable)+'_'+str(label)] = np.where(cat_df[variable]==label,1,0)


Creamos grupos de Train & Test

In [8]:
#separate the other attributes from the predicting attribute
x = datos_meli.drop('price',axis=1)
print(x)
type(x)
x = one_hot_encoder(x,100)

#separte the predicting attribute into Y for model training 
y = datos_meli['price']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=123)

       id_MLA1133566298  id_MLA1132845171  id_MLA1131997388  id_MLA1131488284  \
0                     0                 0                 0                 0   
1                     0                 0                 0                 0   
2                     0                 0                 0                 0   
3                     0                 0                 0                 0   
4                     0                 0                 0                 0   
...                 ...               ...               ...               ...   
20782                 0                 0                 0                 0   
20783                 0                 0                 0                 0   
20784                 0                 0                 0                 0   
20785                 0                 0                 0                 0   
20786                 0                 0                 0                 0   

       id_MLA1131377170  id

Entrenamos el modelo

In [24]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', alpha = 10, subsample= 0.7999999999999999,
n_estimators= 500, max_depth =5, learning_rate= 0.1, colsample_bytree= 0.7999999999999999, colsample_bylevel= 0.4)

xg_reg.fit(X_train,y_train)

[21:15:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=0.4,
             colsample_bynode=1, colsample_bytree=0.7999999999999999,
             enable_categorical=False, gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=500, n_jobs=8,
             num_parallel_tree=1, objective='reg:linear', predictor='auto',
             random_state=0, reg_alpha=10, reg_lambda=1, scale_pos_weight=1,
             subsample=0.7999999999999999, tree_method='exact',
             validate_parameters=1, verbosity=None)

Miramos resultados

In [25]:
y_pred = xg_reg.predict(X_test)

print('Mean Absolute Error:', round(metrics.mean_absolute_error(y_test, y_pred)/1000,2))
print('Mean Squared Error:', round(metrics.mean_squared_error(y_test, y_pred)/1000,2))
print('Root Mean Squared Error:', round(np.sqrt(metrics.mean_squared_error(y_test, y_pred))/1000,2))


Mean Absolute Error: 1237.42
Mean Squared Error: 3664054077.01
Root Mean Squared Error: 1914.17


In [ ]:
# Miro la importancia de las variables
importance = xg_reg.feature_importances_
x.columns
figure(figsize=(15, 4.5), dpi=80,)
plt.bar(x.columns, xg_reg.feature_importances_)
plt.show()

Optimizamos parametros

In [ ]:
########################################## GridSearchCV
from sklearn.model_selection import GridSearchCV

params = { 'max_depth': [3,6,10],
           'learning_rate': [0.01, 0.05, 0.1],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

xgbr = xgb.XGBRegressor(seed = 20)
clf_GS = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring = 'neg_mean_absolute_error'
                   verbose=1)
clf_GridS.fit(X_train, y_train)
print("Best parameters:", clf.best_params_)

y_pred_test = clf_GridS.best_estimator_(X_test)
y_pred_train = clf_GridS.best_estimator_(X_train)

print('Root Mean Squared Error:', round(np.sqrt(metrics.mean_squared_error(y_test, y_pred_test))/1000000,2))
print('Root Mean Squared Error:', round(np.sqrt(metrics.mean_squared_error(y_train, y_pred_train))/1000000,2))



print("Lowest RMSE: ", round(((-clf_GridS.best_score_)**(1/2.0)/1000000),2))
print("Lowest MSE: ", round((-clf_GridS.best_score_/1000000),2))



In [ ]:
########################################## RandomSearch

params = { 'max_depth': [3, 5, 6, 10, 15, 20],
           'learning_rate': [0.01, 0.1, 0.2, 0.3],
           'subsample': np.arange(0.5, 1.0, 0.1),
           'colsample_bytree': np.arange(0.4, 1.0, 0.1),
           'colsample_bylevel': np.arange(0.4, 1.0, 0.1),
           'n_estimators': [100, 500, 1000]}

xgbr = xgb.XGBRegressor(seed = 20)
clf_RandomS = RandomizedSearchCV(estimator=xgbr,
                         param_distributions=params,
                         scoring='neg_mean_squared_error',
                         n_iter=100,
                         verbose=1)
clf_RandomS.fit(X_train, y_train)

print("Best parameters:", clf_RandomS.best_params_)
print("Lowest RMSE: ", round(((-clf_RandomS.best_score_)**(1/2.0)/1000000),2))
print("Lowest MSE: ", round((-clf_RandomS.best_score_/1000000),2))